In [1]:
#ini-iPython
import os
os.chdir("/home/vinicius/Dropbox/Projeto Mestrado/Codigos/RoboFEI-HT_Debug/AI/Visual_Memory") #Executando na pasta Vision
import sys
sys.path.append("./iPython")
#end-iPython

---- List of execution parameters ----

In [2]:
#des-iPython import argparse

In [3]:
#des-iPython parser = argparse.ArgumentParser(
#des-iPython     description="Visual Memory",
#des-iPython     epilog= "Responsável por rastrear todos os objetos detectáveis pela visão.\\" \
#des-iPython             "Responsible for tracking all objects detectable by vision."
#des-iPython )

#des-iPython parser.add_argument(
#des-iPython     "--robot_numbers", # Full name
#des-iPython     "--rn", # Abbreviation for the name
#des-iPython     type = int, # Type variable
#des-iPython     help = "Número de robôs que serão rastreados.\\" \
#des-iPython            "Number of robots to be tracked." # Description of the variable
#des-iPython )

#des-iPython parser.add_argument(
#des-iPython     "--frequency_of_execution", # Full name
#des-iPython     "--f", # Abbreviation for the name
#des-iPython     type = float, # Type variable
#des-iPython     help = "Frequência de execução do programa, caso seja zero ira rodar sem interrupções.\\" \
#des-iPython            "Frequency of program execution, if zero will run without interruptions." # Description of the variable
#des-iPython )

#des-iPython args = parser.parse_args()

#ini-iPython
class test():
    robot_numbers = None
args = test()
#end-iPython

---- Imports ----

In [4]:
# Libraries to be used.
import sys
sys.path.append("./include")
sys.path.append("./src")

In [5]:
# The standard libraries used in the Visual Memory System.
import numpy as np
import os
import time

In [6]:
# Used class developed by RoboFEI-HT
from MainFunctions import * # Declaration the main functions.
from Blackboard import * # Class used to access shared memory.
from Robots import * # Class used to track robot.
from Landmark import * # Class used to track landmark.
from Ball import * # Class used to track ball.
from Speeds import * # Classes used to manage robot speed values.
from ConfigIni import * # Used to read file "config.ini".

---- Main Code ----

Starting processes

In [7]:
killedProcess() # Recognize external kill

In [8]:
# Default values
os.system('rm ./Data/config.ini') #debug-iPython
parameters = {
    "robot_numbers": 8,
    "frequency_of_execution": 0,
}

# Reading "config.ini" values
conf = ConfigIni("Basic", "Settings")
parameters = conf.read(parameters)
if args.robot_numbers != None:
    parameters["robot_numbers"] = args.robot_numbers

if args.frequency_of_execution != None:
    parameters["frequency_of_execution"] = args.frequency_of_execution

if parameters['frequency_of_execution'] != 0:
    parameters['frequency_of_execution'] = 1.0/parameters['frequency_of_execution']

print "Parâmetros:", parameters, "\n" #debug-iPython

# List with velocities for each movement of the robot.
speeds = Speeds( )

mem = Blackboard( ) # Creating shared memory

land = Landmark(speeds) # Creating landmark object

ball = Ball(speeds) # Creating landmark object

# Creating robot objects
robots = []
newrobots = []
for __ in xrange(parameters["robot_numbers"]):
    newrobots.append(Robots(speeds))
print "Qnt. robôs:", len(newrobots) #debug-iPython

Parâmetros: {'robot_numbers': 8, 'frequency_of_execution': 0} 

Qnt. robôs: 8


Main loop

In [9]:
#des-iPython while True:
#ini-tab
try:
    # start timestamp
    start = time.time()
    
    # Reading data about the landmark.
    mem.write_float("VISION_LAND_TAG", 2) #debug-iPython
    taglandmark = mem.read_float("VISION_LAND_TAG")
    print "taglandmark:", taglandmark #debug-iPython
    
    if taglandmark != 0:
        print "Lendo valores de Landmarks" #debug-iPython
        # Read the values sent
        datalandmarks = readDataLandmarks(taglandmark, mem)
        print "Dados landmarks:", datalandmarks #debug-iPython
        speeds.update(land.update(datalandmarks))
        
    else:
        print "Sem valores de Landmarks" #debug-iPython
        land.prediction( )
    
    datarobots = readDataRobots(mem, parameters["robot_numbers"])
    
#     if datarobots == []:
        
    # stop timestamp
    diff = time.time() - start
    print 'Tempo desejado para espera:', parameters["frequency_of_execution"], 's' #debug-iPython
    print 'Tempo de execução:', diff*1000, 'ms' #debug-iPython
    
    if parameters["frequency_of_execution"] - diff > 0:
        print 'Esperando:', (parameters["frequency_of_execution"] - diff)*1000, 'ms' #debug-iPython
        time.sleep(parameters["frequency_of_execution"] - diff)
        print 'Tempo esgotado' #debug-iPython
    
except KeyboardInterrupt:
    os.system("clear") # Cleaning terminal
    print "Keyboard interrupt detected"
    #des-iPython break
except VisualMemoryException as e:
    #edes-iPython break
    pass
#end-tab -- Fim do loop

taglandmark: 2.0
Lendo valores de Landmarks
Dados landmarks: [2.0, 1.0, 5.0, 0.20000000298023224]
Tempo desejado para espera: 0 s
Tempo de execução: 4.11987304688 ms


Finishing processes

In [10]:
# Saving config values
if parameters['frequency_of_execution'] != 0:
    parameters['frequency_of_execution'] = 1.0/parameters['frequency_of_execution']
conf.finalize(parameters)

In [11]:
#ini-iPython

# Testes

In [12]:
import time

In [13]:
start = time.time()

In [14]:
stop = time.time()
stop - start

0.02954697608947754

In [15]:
time.ctime(start)

'Mon Aug 21 14:45:01 2017'

In [16]:
#end-iPython